In [2]:
# Imports needed for the script
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls

from sklearn import tree
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from IPython.display import Image as PImage
from subprocess import check_call
from PIL import Image, ImageDraw, ImageFont
import xgboost as xgb

In [11]:

#Laoding the data

train = pd.read_csv('train.csv')
                    
test = pd.read_csv('test.csv')

# Store passanger IDs

PassengerId = test['PassengerId']


train.head(3)
                    

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


In [24]:
#make copy of the main data 
original_train = train.copy()

full_data = [train, test]

# tells weather the passenger has a cabin on the Titanic
train['Has_Cabin'] = train["Cabin"].apply(lambda x: 0 if type(x) == float else 1 )
test['Has_Cabin'] = test["Cabin"].apply(lambda x: 0 if type(x) == float else 1)


for dataset in full_data:
    dataset['FamilySize'] = dataset['SibSp'] + dataset['Parch'] + 1
    

#create new feature IsAlone from family size

for dataset in full_data:
    dataset['IsAlone'] = 0
    dataset.loc[dataset['FamilySize'] == 1, 'IsAlone'] = 1
    
for dataset in full_data:

    dataset['Embarked'] = dataset['Embarked'].fillna('S')
    
for dataset in full_data:
    dataset["Fare"] = dataset['Fare'].fillna(train['Fare'].median())


    
# remove all nulls in the age column

for dataset in full_data:
    age_avg = dataset['Age'].mean()
    age_std = datset['Age'].std()
    age_null_count = dataset["Age"].isnull().sum()
    age_null_random_list = np.random.randint(age_avg - age_std, age_avg + age_std, size = age_null_count)
    
    dataset.loc[np.isnan(dataset['Age']), 'Age'] = age_null_random_list
    dataset['Age'] = dataset['Age'].astype(int)
    
print(age_null_random_list)

[40 41 27 23 33 23 17 21 35 17 25 36 36 20 17 19 42 27 34 28 29 36 29 27
 38 34 42 36 20 34 16 22 18 16 28 25 29 38 39 33 38 29 39 27 34 18 24 17
 40 34 37 16 28 23 41 21 21 21 38 17 28 21 19 28 30 24 31 30 36 21 33 29
 31 32 18 32 22 21 31 26 24 39 19 35 41 38]
